In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [4]:
data=pd.read_csv("checkins.csv")

In [6]:
data=data[:100000]

In [9]:
from sklearn.cluster import MeanShift

In [17]:
data=data[["latitude","longitude"]]

In [18]:
model=MeanShift(bandwidth=0.1)

In [19]:
preds=model.fit_predict(data)

In [21]:
preds.shape

(100000L,)

In [26]:
model.cluster_centers_.shape

(3230L, 2L)

In [37]:
model.labels_

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [54]:
claster_centers_df=pd.DataFrame(model.cluster_centers_)
claster_centers_df.columns=["cent_latitude","cent_longitude"]

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 3 while Y.shape[1] == 2

In [55]:
data["cluster"]=preds

In [56]:
cluster_size=pd.DataFrame(data.pivot_table(index="cluster",aggfunc="count",values="latitude"))

In [62]:
cluster_size.columns=["clust_size"]

In [65]:
cluster_df=claster_centers_df.join(cluster_size)
cluster_df=cluster_df[cluster_df.clust_size>15]

In [67]:
def  get_distance(lat1,lon1,lat2,lon2):
    return ((lat1-lat2)**2+(lon1-lon2)**2)**0.5

In [68]:
office_coordinates=[[33.751277, -118.188740],
                   [25.867736, -80.324116],
                   [51.503016, -0.075479],
                   [52.378894, 4.885084],
                   [39.366487, 117.036146 ],
                   [-33.868457, 151.205134]]

In [69]:
def get_min_distance(lat,lon):
    min_dist=None
    for (of_lat,of_lon)in office_coordinates:
        dist=get_distance(lat,lon,of_lat,of_lon)
        if min_dist is None or dist<min_dist:
            min_dist=dist
    return min_dist

In [70]:
cluster_df["min_distance"]=map(get_min_distance,cluster_df.cent_latitude,cluster_df.cent_longitude)

In [74]:
cluster_df.sort_values("min_distance")[:20]

,cent_latitude,cent_longitude,clust_size,min_distance
413,-33.860630,151.204776,28,0.007835
375,52.372964,4.892317,31,0.009353
410,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
29,25.785812,-80.217938,564,0.134109
165,25.705350,-80.283429,80,0.167406
92,26.010098,-80.199991,138,0.188876
87,33.888325,-118.048928,100,0.195779
42,33.872986,-118.362091,384,0.211811
